## Background
In this homework we will practice with interactive visualization, which is the key ingredient of many successful viz (especially when it comes to infographics).
You will be working with the P3 database of the [SNSF](http://www.snf.ch/en/Pages/default.aspx) (Swiss National Science Foundation).
As you can see from their [entry page](http://p3.snf.ch/), P3 already offers some ready-made viz, but we want to build a more advanced one for the sake
of quick data exploration. Therefore, start by [downloading the raw data](http://p3.snf.ch/Pages/DataAndDocumentation.aspx) (just for the Grant Export), and read carefully
the documentation to understand the schema. Install then [Folium](https://github.com/python-visualization/folium) to deal with geographical data (*HINT*: it is not
available in your standard Anaconda environment, therefore search on the Web how to install it easily!) The README file of Folium comes with very clear examples, and links 
to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find a TopoJSON file with the 
geo-coordinates of each Swiss canton (which can be used as an overlay on the Folium maps).

## Assignment
1. Build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton.
To do so, you will need to use the provided TopoJSON file, combined with the Choropleth map example you can find in the Folium README file.

*HINT*: the P3 database is formed by entries which assign a grant (and its approved amount) to a University name. Therefore you will need a smart strategy to go from University
to Canton name. The [Geonames Full Text Search API in JSON](http://www.geonames.org/export/web-services.html) can help you with this -- try to use it as much as possible
to build the canton mappings that you need. For those universities for which you cannot find a mapping via the API, you are then allowed to build it manually -- feel free to stop 
by the time you mapped the top-95% of the universities. I also recommend you to use an intermediate viz step for debugging purposes, showing all the universties as markers in your map (e.g., if you don't select the right results from the Geonames API, some of your markers might be placed on nearby countries...)

2. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in research funding
between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

*HINT*: for those cantons cut through by the Röstigraben, [this viz](http://p3.snf.ch/Default.aspx?id=allcharts) can be helpful!


# Data fields & description 

* __Project Number__ -- uid
* __Project Title__ -- Name of the project. Mandatory field.
* __Project Title English__ -- Name of the project in English. Optional field.
* __Reponsible Applican__ -- Person who submitted the application to the SNSF. Mandatory field.
* __Funding Instrument__ -- Funding scheme
* __Funding Instrument Hierarchy__ -- Top Level Hierarchy Funding Instrument.
* __Institution__ -- Research institution where the project will largely be carried out. Typically, this is the institution or specific lab where the responsible applicant works. "This field is not structured, but its content is consolidated internally." There is no link in the database to the field University.
* __University__ -- This is the institution where the project will largely be carried out according to the application. Pick list. This field is only filled if the research is carried out at a Swiss institution, otherwise the field remains blank. In the case of mobility fellowships, it is generally left empty.
* __Discipline Number__ -- Number of the discipline. Only the main discipline from a list of disciplines supplied by the SNSF.
* __Discipline Name__ -- Name of the discipline. 
* __Discipline Name Hierarchy__ -- Top level of discipline.
* __Start Date__ -- Date on which the project starts.
* __End Date__ -- Actual end date of the project (updated if changed).
* __Approved Amount__ -- Approved amount, changed if the award is corrected. This amount is not indicated in the case of mobility fellowships.
* __Keywords__ -- unstructured field.
* __Abstract (Project)__ -- The scientific abstract of the research project. The researchers are responsible for the contents.
* __Lay Summary - Lead__ -- Summarise the most important information in the Lay Summary.
* __Lay Summary__ -- Summary of a research project aimed at laypeople. 

In [323]:
import folium
import pickle

In [324]:
map_osm = folium.Map(location=[46.92287,8.3829913], zoom_start=8)
map_osm

In [325]:
folium.TopoJson(open('ch-cantons.topojson.json'),
                'objects.cantons',
                name='topojson'
               ).add_to(map_osm)
map_osm

In [326]:
wc = pickle.load(open('all_cantons.p','rb')).reset_index()
cantons_geo = './ch-cantons.topojson.json'

map_osm.choropleth(
    geo_path=cantons_geo, 
    data=wc,
    columns=['Canton', 'Approved Amount'],
    topojson='objects.cantons',
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.5,
    threshold_scale=[0, 1, 10, 100, 1000, 5000],
    legend_name='Grant amount (MCHF)',
    reset=True
)

In [327]:
map_osm

In [328]:
unis = pickle.load(open('uni_geo_infos.p','rb'))
for key, val in unis.items():
    if val is not None:
        popup_str = key
        folium.Marker([val['location']['lat'], val['location']['lng']], popup=popup_str).add_to(map_osm)

In [329]:
map_osm

In [330]:
import pandas as pd

DE = 0
FR = 1
IT = 2

cantons_lang = {
    'ZH' : DE,
    'BE' : DE,
    'LU' : DE,
    'UR' : DE,
    'SZ' : DE,
    'OW' : DE,
    'NW' : DE,
    'GL' : DE,
    'ZG' : DE,
    'FR' : DE,
    'SO' : DE,
    'BS' : DE,
    'BL' : DE,
    'SH' : DE,
    'AR' : DE,
    'AI' : DE,
    'SG' : DE,
    'GR' : DE,
    'AG' : DE,
    'TG' : DE,
    'TI' : IT,
    'VD' : FR,
    'VS' : FR,
    'NE' : FR,
    'GE' : FR,
    'JU' : FR
}


rosti_pd = pd.DataFrame.from_dict(cantons_lang, orient='index')
rosti_pd.reset_index(inplace=True)
rosti_pd.rename(index=str, columns={'index': "Canton", 0:"Language"}, inplace=True)
rosti_pd

,Canton,Language
0,NE,1
1,AI,0
2,AG,0
3,VD,1
4,VS,1
5,ZH,0
6,JU,1
7,OW,0
8,BL,0
9,SZ,0


In [331]:
cantons_geo = './ch-cantons.topojson.json'
map_rosti = folium.Map(location=[46.92287,8.3829913], zoom_start=8)

map_rosti.choropleth(
    geo_path=cantons_geo, 
    data=rosti_pd,
    columns=['Canton', 'Language'],
    topojson='objects.cantons',
    key_on='feature.id',
    fill_color='YlOrRd',
    threshold_scale=[0, 1, 2],
    line_opacity=0.5
)
map_rosti

In [332]:
folium.CircleMarker([46.92287,8.3829913],
                    radius=50000,
                    popup='Funding Swiss-German universities',
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map_rosti)
folium.CircleMarker([46.5190557,6.5667576],
                    radius=40000,
                    popup='Funding Swiss-French universities',
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map_rosti)

In [333]:
map_rosti